In [32]:
from sklearn.svm import SVR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import data_processing as pf
import altair as alt
from sklearn.metrics import mean_squared_error, r2_score

In [33]:
alt.renderers.enable('default')
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [55]:
googl = yf.Ticker('GOOGL')
amzn = yf.Ticker('AMZN')
aapl = yf.Ticker('AAPL')
fb = yf.Ticker('FB')
msft = yf.Ticker('MSFT')
tsla = yf.Ticker('TSLA')

aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

In [78]:
stock_dfs = [amzn_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_close_setup(stock_df, 1)

In [72]:
def combiner(stock_dfs):
    combine_df = stock_dfs[0]
    i = 1
    while i < len(stock_dfs):
        combine_df = pd.concat([combine_df, stock_dfs[i]])
        i += 1
    
    combine_df.sort_values(by=['Date'], inplace=True)
    return combine_df

In [79]:
combine_df = combiner(stock_dfs)
X = combine_df.iloc[:,:-1]
y = combine_df.iloc[:,-1:]


In [80]:
from sklearn.model_selection import train_test_split

split_mark = int(len(combine_df)-(365*len(stock_dfs)))
X_train = X.head(split_mark)
X_test = X.tail(len(combine_df) - split_mark)
y_train = y.head(split_mark)
y_test = y.tail(len(combine_df) - split_mark)



In [82]:
from sklearn.preprocessing import StandardScaler

sc =  StandardScaler()

X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)


In [83]:
svr_rbf = SVR(kernel='rbf', C=1e8, gamma=1e-8,epsilon=0.001) 
#pol_svr = SVR(kernel = 'poly', C=1000.0, degree=10)
svr_rbf.fit(X_train_scaled,y_train)
#pol_svr.fit(X_train,y_train)
svr_close_pred = svr_rbf.predict(X_test_scaled)
svr_train_pred = svr_rbf.predict(X_train_scaled)

#pol_close_pred = pol_svr.predict(X_test)
#pol_train_pred = pol_svr.predict(X_train)

/home/matthewarinanta/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [12]:
train_score = r2_score(y_train, svr_train_pred)
print("Training R2 Score: " + str(train_score))



Training R2 Score: 0.9973964022452387


In [13]:
model_score = r2_score(y_test.dropna(), svr_close_pred[:len(y_test.dropna())])
print("R2 Score: " + str(model_score))



R2 Score: 0.9609414451050441


In [16]:
stock_dfs[0].drop(stock_dfs[0].tail(1).index,inplace=True)

In [84]:
columns = ['Amazon Price', 'Amazon Prediction']

single_split_mark = int(len(stock_dfs[0])-365)
results_df = pd.DataFrame(columns=columns, index=stock_dfs[0].tail(len(stock_dfs[0])-single_split_mark).index)

i = 0
for stock_df in stock_dfs:
     single_split_mark = int(len(stock_df)-365)
     results_df[columns[i]] =  stock_df.iloc[:,-1:].tail(len(stock_df) - single_split_mark)
        
     X_all = sc.transform(stock_df.iloc[:,:-1].tail(len(stock_df) - single_split_mark))
     results_df[columns[i+1]] = svr_rbf.predict(X_all)
     i += 2
            
results_df.reset_index(inplace=True)
results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
line_plot = alt.Chart(results_df).mark_line().encode(
    x = 'Date',
    y = 'Price',
    color = 'Company'
    )

line_plot

alt.Chart(...)

In [92]:
stock_dfs = [googl_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_close_setup(stock_df, 1)

combine_df = combiner(stock_dfs)
X = combine_df.iloc[:,:-1]
y = combine_df.iloc[:,-1:]

split_mark = int(len(combine_df)-(365*len(stock_dfs)))
X_train = X.head(split_mark)
X_test = X.tail(len(combine_df) - split_mark)
y_train = y.head(split_mark)
y_test = y.tail(len(combine_df) - split_mark)

sc =  StandardScaler()

X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

svr_rbf = SVR(kernel='rbf', C=1e8, gamma=1e-8,epsilon=0.001) 
#pol_svr = SVR(kernel = 'poly', C=1000.0, degree=10)
svr_rbf.fit(X_train_scaled,y_train)
#pol_svr.fit(X_train,y_train)
svr_close_pred = svr_rbf.predict(X_test_scaled)
svr_train_pred = svr_rbf.predict(X_train_scaled)

/home/matthewarinanta/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [94]:
columns = ['Google Price', 'Google Prediction']

single_split_mark = int(len(stock_dfs[0])-365)
results_df = pd.DataFrame(columns=columns, index=stock_dfs[0].tail(len(stock_dfs[0])-single_split_mark).index)

i = 0
for stock_df in stock_dfs:
     single_split_mark = int(len(stock_df)-365)
     results_df[columns[i]] =  stock_df.iloc[:,-1:].tail(len(stock_df) - single_split_mark)
        
     X_all = sc.transform(stock_df.iloc[:,:-1].tail(len(stock_df) - single_split_mark))
     results_df[columns[i+1]] = svr_rbf.predict(X_all)
     i += 2
            
results_df.reset_index(inplace=True)
results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
line_plot = alt.Chart(results_df).mark_line().encode(
    x = 'Date',
    y = 'Price',
    color = 'Company'
    )
line_plot

alt.Chart(...)

In [96]:
stock_dfs = [msft_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_close_setup(stock_df, 1)

combine_df = combiner(stock_dfs)
X = combine_df.iloc[:,:-1]
y = combine_df.iloc[:,-1:]

split_mark = int(len(combine_df)-(365*len(stock_dfs)))
X_train = X.head(split_mark)
X_test = X.tail(len(combine_df) - split_mark)
y_train = y.head(split_mark)
y_test = y.tail(len(combine_df) - split_mark)

sc =  StandardScaler()

X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

svr_rbf = SVR(kernel='rbf', C=1e8, gamma=1e-8,epsilon=0.001) 
#pol_svr = SVR(kernel = 'poly', C=1000.0, degree=10)
svr_rbf.fit(X_train_scaled,y_train)
#pol_svr.fit(X_train,y_train)
svr_close_pred = svr_rbf.predict(X_test_scaled)
svr_train_pred = svr_rbf.predict(X_train_scaled)

columns = ['Microsoft Price', 'Microsoft Prediction']

single_split_mark = int(len(stock_dfs[0])-365)
results_df = pd.DataFrame(columns=columns, index=stock_dfs[0].tail(len(stock_dfs[0])-single_split_mark).index)

i = 0
for stock_df in stock_dfs:
     single_split_mark = int(len(stock_df)-365)
     results_df[columns[i]] =  stock_df.iloc[:,-1:].tail(len(stock_df) - single_split_mark)
        
     X_all = sc.transform(stock_df.iloc[:,:-1].tail(len(stock_df) - single_split_mark))
     results_df[columns[i+1]] = svr_rbf.predict(X_all)
     i += 2
            
results_df.reset_index(inplace=True)
results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
line_plot = alt.Chart(results_df).mark_line().encode(
    x = 'Date',
    y = 'Price',
    color = 'Company'
    )
line_plot

/home/matthewarinanta/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


alt.Chart(...)